# Django

# Librerias

In [1]:
from django.db import models

# Intro


## Request y Respuestas

In [ ]:
### Anaconda CMD: crear proyecto

In [21]:
# cambiar a directorio back (desarrollo)
cd  C:/Users/Diego/Documents/GitHub/tech_skills/back


C:\Users\Diego\Documents\GitHub\tech_skills\back


In [ ]:
# ejecutar comando desde Anaconda CMD
# crear directorio mysite
django-admin startproject mysite 


In [ ]:
# cambiar a directorio del proyecto
cd  C:/Users/Diego/Documents/GitHub/tech_skills/back/mysite
python manage.py runserver


In [ ]:
# cambiar a directorio del proyecto
cd C:/Users/Diego/Documents/GitHub/tech_skills/back/mysite

# crear aplicacion Polls 
python manage.py startapp polls


## Crear View

In [ ]:
# en directorio mysite/polls/views.py
# crear vista de indice simple

# crear Polls index
from django.http import HttpResponse

# crear Polls index para aplicacion
def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")

In [ ]:
# en directorio mysite/polls/urls.py

# crear patron URL a vista de indice

from django.urls import path

from . import views

urlpatterns = [
    path("", views.index, name="index"),
]

In [ ]:
# en directorio mysite/mysite/urls.py

# URL configuration for mysite project.

from django.contrib import admin
from django.urls import path, include

# agregar patron URL de aplicacion polls en directorio polls/
# incluir polls.urls
urlpatterns = [
    path("polls/", include("polls.urls")),
    path("admin/", admin.site.urls),
]

## Crear Modelos

In [ ]:
# en directorio mysite externo
# ejecutar comando migrate
# crear tablas (bases de datos) a partir de aplicaciones instaladas
python manage.py migrate



In [ ]:
# en archivo polls/models.py
# crear modelos (clases)

from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)


In [ ]:
# en archivo mysite/settings.py 
# actualizar lista de aplicaciones instaladas
# incluir "polls.apps.PollsConfig"

INSTALLED_APPS = [
    "polls.apps.PollsConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

In [ ]:
# en directorio mysite externo
# ejecutar comando makemigrations polls
# crear migraciones para los modelos nuevos/actualizados
# añadir nuevos modelos a lista de migraciones
python manage.py makemigrations polls

In [ ]:
# en directorio mysite externo
# ejecutar comando migrate
# crear tablas (bases de datos) a partir de aplicaciones instaladas, incluidos los nuevos modelos (tablas)
python manage.py migrate

## Acceder a Shell en Django

In [ ]:
# en directorio mysite externo
# ejecutar comando mmanage.py shell
# abrir shell en django
python manage.py shell

In [ ]:

# ejecutar desde shell django
>>> from polls.models import Choice, Question  # Import the model classes we just wrote.

# No questions are in the system yet.
>>> Question.objects.all()
<QuerySet []>

# Create a new Question.
# Support for time zones is enabled in the default settings file, so
# Django expects a datetime with tzinfo for pub_date. Use timezone.now()
# instead of datetime.datetime.now() and it will do the right thing.
>>> from django.utils import timezone
>>> q = Question(question_text="What's new?", pub_date=timezone.now())

# Save the object into the database. You have to call save() explicitly.
>>> q.save()

# Now it has an ID.
>>> q.id
1

# Access model field values via Python attributes.
>>> q.question_text
"What's new?"
>>> q.pub_date
datetime.datetime(2012, 2, 26, 13, 0, 0, 775217, tzinfo=datetime.timezone.utc)

# Change values by changing the attributes, then calling save().
>>> q.question_text = "What's up?"
>>> q.save()

# objects.all() displays all the questions in the database.
>>> Question.objects.all()
<QuerySet [<Question: Question object (1)>]>

## Crear usuario Admin

In [ ]:
# en directorio mysite externo
# ejecutar comando manage.py createsuperuser
# crear usuario admin (super-user) 
python manage.py createsuperuser

# ingresar credenciales usuario admin
Username: admin
Email address: admin@example.com
Password: **********
Password (again): *********

Superuser created successfully.

In [ ]:
# en directorio mysite externo
# ejecutar comando manage.py runserver
# iniciar servidor de desarrollo
python manage.py runserver



In [ ]:
# cambiar pool app para modificar por usuario admin
# en archivo polls/admin.py
from django.contrib import admin

from .models import Question

admin.site.register(Question)


## Crear Views


In [ ]:
# en archivo  polls/views.py
# agregar views (clases)
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)


def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)


def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)


In [ ]:
# en archivo  polls/urls.py
# agregar paths de clases agregadas en archivo view a lista de  urlpatterns

urlpatterns = [
    # ex: /polls/
    path("", views.index, name="index"),
    # ex: /polls/5/
    path("<int:question_id>/", views.detail, name="detail"),
    # ex: /polls/5/results/
    path("<int:question_id>/results/", views.results, name="results"),
    # ex: /polls/5/vote/
    path("<int:question_id>/vote/", views.vote, name="vote"),
]


In [ ]:
# en archivo  polls/views.py
# modificar vista index

from django.http import HttpResponse

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:5]
    output = ", ".join([q.question_text for q in latest_question_list])
    return HttpResponse(output)



In [ ]:
# crear directorio:  /mysite/polls/templates/polls
# crear archivo index.hmtl  en  directorio /mysite/polls/templates/polls


{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}

In [ ]:
# actualizar  archivo  polls/views.py

from django.http import HttpResponse
from django.template import loader

from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:5]
    template = loader.get_template("polls/index.html")
    context = {
        "latest_question_list": latest_question_list,
    }
    return HttpResponse(template.render(context, request))

## Render

In [ ]:
# en archivo polls/views.py
# actualizar archivo para incluir render


# render(request, "polls/index.html", context)

from django.shortcuts import render

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:5]
    context = {"latest_question_list": latest_question_list}
    return render(request, "polls/index.html", context)

## Raising a 404 error

In [ ]:
# en archivo polls/views.py
# incluir modelo (clase)  detail

def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, "polls/detail.html", {"question": question})


## get_object_or_404()

In [ ]:
# en archivo polls/views.py
# incluir modelo (clase)  detail

from django.shortcuts import get_object_or_404, render
from .models import Question


# ...
def detail(request, question_id):
    
    # mostrar modelo, o generar  Http404 si no existe
    question = get_object_or_404(Question, pk=question_id)
    return render(request, "polls/detail.html", {"question": question})

## URL names

In [ ]:
# en archivo polls/urls.py
# agregar app_name
# en lista de urls (urlpatterns) incluir paths de cada modelo 

from django.urls import path

from . import views

app_name = "polls"
urlpatterns = [
    path("", views.index, name="index"),
    path("<int:question_id>/", views.detail, name="detail"),
    path("<int:question_id>/results/", views.results, name="results"),
    path("<int:question_id>/vote/", views.vote, name="vote"),
]


In [ ]:
# en archivo polls/index.html
# actualizar template 

<li><a href="{% url 'polls:detail' question.id %}">{{ question.question_text }}</a></li>


## Formas y Views generales

In [3]:
# en archivo polls/templates/polls/detail.html
# actualizar detail template
# agregar HTML <form>
<form action="{% url 'polls:vote' question.id %}" method="post">
{% csrf_token %}
<fieldset>
    <legend><h1>{{ question.question_text }}</h1></legend>
    {% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}
    {% for choice in question.choice_set.all %}
        <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}">
        <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br>
    {% endfor %}
</fieldset>
<input type="submit" value="Vote">
</form>


In [ ]:
# en archivo polls/views.py
# agregar modelo vote

from django.http import HttpResponse, HttpResponseRedirect
from django.shortcuts import get_object_or_404, render
from django.urls import reverse

from .models import Choice, Question

# ...
def vote(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST["choice"])
    except (KeyError, Choice.DoesNotExist):
        # Redisplay the question voting form.
        return render(
            request,
            "polls/detail.html",
            {
                "question": question,
                "error_message": "You didn't select a choice.",
            },
        )
    else:
        selected_choice.votes += 1
        selected_choice.save()
        # Always return an HttpResponseRedirect after successfully dealing
        # with POST data. This prevents data from being posted twice if a
        # user hits the Back button.
        return HttpResponseRedirect(reverse("polls:results", args=(question.id,)))



In [ ]:
# en archivo polls/views.py
# agregar modelo results

def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, "polls/results.html", {"question": question})


In [ ]:
# en directorio polls/templates/polls/results.html

<h1>{{ question.question_text }}</h1>

<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }} -- {{ choice.votes }} vote{{ choice.votes|pluralize }}</li>
{% endfor %}
</ul>

<a href="{% url 'polls:detail' question.id %}">Vote again?</a>

## Views Genericas



### Amend URLconf

In [ ]:
# en archivo polls/urls.py
# actualizar lista urlpatterns
from django.urls import path

from . import views

app_name = "polls"
urlpatterns = [
    path("", views.IndexView.as_view(), name="index"),
    path("<int:pk>/", views.DetailView.as_view(), name="detail"),
    path("<int:pk>/results/", views.ResultsView.as_view(), name="results"),
    path("<int:question_id>/vote/", views.vote, name="vote"),
]


### Amend views

In [ ]:
# en archivo polls/views.py
# reemplazar views index, detail, results  por vistas genericas

from django.http import HttpResponseRedirect
from django.shortcuts import get_object_or_404, render
from django.urls import reverse
from django.views import generic

from .models import Choice, Question


class IndexView(generic.ListView):
    template_name = "polls/index.html"
    context_object_name = "latest_question_list"

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by("-pub_date")[:5]


class DetailView(generic.DetailView):
    model = Question
    template_name = "polls/detail.html"


class ResultsView(generic.DetailView):
    model = Question
    template_name = "polls/results.html"


def vote(request, question_id):
    ...  # same as above, no changes needed.
    
    
    

# Model Layer

# View Layer

# Template Layer

# Forms

# Admin

# Seguridad

# Desarrollo